# Динамическое программирование

Динамическое программирование - это часто очень эффективный способ решения алгоритмических задач. Его суть заключается в том, что в процессе вычисления итогового ответа, мы активно используем данные о промежуточных результатах, которые мы уже вычислили.

Самым простым примером, показывающим идею динамического программирования - это вычисление чисел Фибоначчи. 
Наивный алгоритм вычисления i-того числа вызовет эту же функцию для вычисления i-1-го и i-2-го числа, а потом сложит эти результаты.

Проанализируем сколько вычислений нам необходимо будет сделать: первый вызов функции породит еще два вызова. Эти два соответственно еще по 2, то есть 4 вызова. Далее эти функции вызовут 8 функций и так далее пока мы не дойдем до вычисления первого числа фибоначчи. 

Итого: 1 + 2 + 4 + 8 + ... ≈ 2^(n). (Это не точное значение операций - только его порядок). Таким образом сложность вычисления растет экспоненциально и для вычисления, скажем, 20 числа фибоначчи нам потребуется порядка 1 миллиона операций!

Где же именно мы теряем так много времени? Рассмотрим вычисление четвертого числа фибоначчи:

F4 = F3 + F2 = (F1 + F2) + F2.

<img src="img/DynamicProg.png">

Видно, что в процессе вычисление F3 мы уже вычислили значение F2. Однако наш алгоритм после вычисления F3 все равно вызовет функцию, которая вычислит F2 снова. 
И далее, количество таких бесполезных вычислений будет только расти.

Основная идея ускорения - не пересчитывать те значения, которые нам уже известны.

* Для вычисления n-го числа фибоначчи заведем массив из n элементов.
* В первые две ячейки запишем первые два числа фибоначчи. 
* Далее будем идти по массиву и в i-тую ячейку записывать значение суммы i-1 и i-2 ячейки массива. Так как мы идем слева направо, то значения этих ячеек нам будут уже известны.
* Дойдя до конца массива мы запишем в n-ую ячейку n-ое число фибоначчи - а это и есть то, что мы искали.

Таким образом для вычисления ВСЕХ (мы ведь заполнпили весь массив, а значит знаем все числа фибоначчи до n) чисел фибоначчи от 1 до 20 нам потребуется 20 операций. Сравните с наивным алгоритмом, который вычисляет 1 число фибоначчи за около миллион операций.

In [1]:
#include <iostream>
#include <vector>
#include <chrono>

In [2]:
typedef std::chrono::milliseconds Milliseconds;
typedef std::chrono::steady_clock Clock;
typedef Clock::time_point Time;

Time start;
unsigned long t;

(unsigned long) 0


In [3]:
long fib_simple(int i) {
    if(i <= 0) return 0;
    if(i <= 2) return 1;
    return fib_simple(i-1) + fib_simple(i-2);
}

In [4]:
start = Clock::now();
long f1 = fib_simple(50);
std::cout << f1 << std::endl;
t = std::chrono::duration_cast<Milliseconds>(Clock::now() - start).count();
std::cout << t << " миллисекунд" << std::endl;

12586269025
63932 миллисекунд


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fa8b3d476e0


In [5]:
long fib_dynamic(int i) {
    std::vector<long> fibs(i+1);
    fibs[0] = 0;
    fibs[1] = 1;
    fibs[2] = 1;
    for(int j = 3; j <= i; j++){
        fibs[j] = fibs[j-1] + fibs[j-2];
    }
    return fibs[i];
}

In [6]:
start = Clock::now();
long f2 = fib_dynamic(50);
std::cout << f2 << std::endl;
t = std::chrono::duration_cast<Milliseconds>(Clock::now() - start).count();
std::cout << t << " миллисекунд" << std::endl;

12586269025
0 миллисекунд


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fa8b3d476e0


Вычисление числа фибоначчи отработало насколько быстро, что это не отловил системный таймер, в то время как наивный алгоритм работал около минуты.

# Расстояние Левенштейна

Одним из применений динамического программиования является расчет редакционного расстрояния (оно же - расстояние Левенштейна в честь Владимира Иосифовича Левенштейна).

Путь у нас есть два строки S1 и S2. Необходимо сказать, какое минимальное количество операций вставки, удаления и замены символов необходимо, чтобы сделать из строки S1 строку S2.

Пример: Роса и Роза - редакционное расстроение равно 1, так как необходимо всего лишь заменить букву c на з. Лес и Лось - в данном примере расстояние будет равно 2, так как необходимо заменить е на о, а после этого еще вставить мягкий знак на конец слова.

Наиболее известный алгоритм расчета расстрояния Левенштейна является алгоритм Вагнера-Фишера. Суть его в следующем:

Умея вычислять расстрояние Левенштейна для подстрок, мы можем вычислить рекуретно расстрояние для строк целиком.

Путь D(i, j) - расстрояние между подстроками S1[1:i] и S2[1:j]. Тогда D(N, M), где N и M - это длины строк S1 и S2 соответственно, и есть та величина, которую мы ищем.

Определим некоторые факты про функцию D:
* D(0, 0) = 0. Расстояние между пустыми строками, очевидно, 0.
* D(i, 0) = i, D(0, j) = j. То есть, самый быстрый способ получить из пустой строки нужное слово - вставить каждую букву этого слова. Все остальные операции, очевидно, будут только увеличивать расстрояние.
* Если S1[i] = S2[j], то D(i, j) = D(i-1, j-1). Это логично, так как если символы совпали, то не нужно с ним производить никаких операций и в итоге расстрояние остается таким же как и без этого символа.
* Если же S1[i] != S2[j], то тогда возможны следующие варианты:
 * Удалить этот символ. Тогда расстояние будет D(i-1, j) + 1, так как мы убрали 1 символ из первой строки и сделали одну операцию удаления.
 * Добавить символ. Тогда, аналогично, расстояние будет D(i, j-1) + 1.
 * Заменить символ. Тогда расстрояние будет D(i-1, j-1) + 1. Этот случай похож на тот, когда символы совпали, только теперь мы еще потратили 1 операцию на замену.

Этих фактов нам хватает для того, чтобы составить алгоритм, вычисляющий редакционное расстрояние.

* Создадим двумерный массив M на N. В i, j -той ячейке будет хранится значение D(i, j).
* Проинициализируем начальные данные, которые нам известны - запомним D(0, 0), D(0, j), D(i, 0);
* Далее будем идти от 1, 1 до M, N и вычислять соответствующие значения D
 * Если S1[i]=S2[j], то запишем D(i-1, j-1)
 * Если нет, то запишем минимальное расстояние, получающееся от различных операций, то есть D(i, j) = min(D(i-1, j), D(i, j-1), D(i-1, j-1)) + 1.
* После окончания в ячейке M, N будет лежать D(M, N), то есть искомая величина

<img src="img/Lev.png">

В данном примере, мы сначала удалили символ Д, далее заменили букву Г на Р и в конце вставили букву Т. Таким образом из слова ДАГЕСТАН мы получили слово АРЕСТАНТ. Итого расстояние - 3, что можно видеть в правой нижней ячейке.

In [7]:
int Levenstain(std::string S1, std::string S2) {
    auto minVal = [](int a, int b, int c) {return std::min(a, std::min(b, c));}; // минимум из трех
    int M = S1.size();
    int N = S2.size();
    std::vector< std::vector<int> > D(M+1, std::vector<int>(N+1)); // матрица
    
    D[0][0] = 0;
    for(int i = 0; i <= M; i++) D[i][0] = i;
    for(int j = 0; j <= N; j++) D[0][j] = j;
    
    for(int i = 1; i <= M; i++) {
        for(int j = 1; j <= N; j++) {
            if(S1[i] == S2[j]){
                D[i][j] = D[i-1][j-1];
            } 
            else {
                D[i][j] = minVal(
                    D[i-1][j],
                    D[i][j-1],
                    D[i-1][j-1]
                ) + 1;
            }
        }
    }
    
    return D[M][N];
}

In [8]:
std::cout << Levenstain("Les", "Loso") << std::endl;

2


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fa8b3d476e0


In [9]:
std::cout << Levenstain("abba", "abba") << std::endl;

0


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fa8b3d476e0


In [10]:
std::cout << Levenstain("POLYNOMIAL", "EXPONENTIAL") << std::endl;

6


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fa8b3d476e0


# Расстрояние Дамерау-Левенштейна

Расстояние Левенштейна - это очень удобный способ сравнения строк на схожесть. Однако на практике часто лучше использовать модифицированную метрику - расстрояние Дамерау-Левенштейна.

Это расстрояние является расширением классического расстояния Левенштейна: теперь, помимо вставки, удаления и замены, добавляется операция транспозиции - замена местами двух соседних символов. Это удобно, при решении задачи исправления текста от опечаток, так как очень большое количество опечаток - это именно транспозиция двух символов.

К примеру, расстояние дамерау-левенштейна между "привет" и "пирвет" всего 1, так как необходимо всего лишь поменять местами "р" и "и". 

Алгоритм вычисления данного редакционного расстояния аналогичен обычному Левенштейну - необходимо лишь добавить обработку случая, когда выгоднее сделать транспозицию.

* Если S1[i-1] = S2[j] и S1[i] = S2[j-1] (возможна транспозиция), то мы опять же можем вставить, удалить и заменить, как и в Левенштейне, однако теперь мы еще можем сделать транспозицию. В таком случае расстояние будет равно D(i-2, j-2) + 1, так как мы восстановили строку до последних двух символов и далее за одну операцию поменяли их местами.
* Если же траспозиция невозможна, то формула превращается в ту, что была в обычном Левинштейне.

In [11]:
int Damerau_Levenstain(std::string S1, std::string S2) {
    auto minVal3 = [](int a, int b, int c) {return std::min(a, std::min(b, c));}; // минимум из трех
    auto minVal4 = [](int a, int b, int c, int d) {return std::min(std::min(a, b), std::min(c, d));}; // из четырех
    int M = S1.size();
    int N = S2.size();
    std::vector< std::vector<int> > D(M+1, std::vector<int>(N+1)); // матрица
    
    D[0][0] = 0;
    for(int i = 0; i <= M; i++) D[i][0] = i;
    for(int j = 0; j <= N; j++) D[0][j] = j;
    
    for(int i = 1; i <= M; i++) {
        for(int j = 1; j <= N; j++) {
            if(i > 1 and j > 1 and S1[i-1] == S2[j] and S1[i] == S2[j-1]) { // возможна транспозиция
                D[i][j] = minVal4(
                    D[i-1][j] + 1,
                    D[i][j-1] + 1,
                    D[i-1][j-1] + (int)(S1[i] != S2[j]), // добавим 1 операцию, если не совпали и нужно заменить
                    D[i-2][j-2] + 1
                );
            } else { // классический левенштейн
                if(S1[i] == S2[j]){
                    D[i][j] = D[i-1][j-1];
                } 
                else {
                    D[i][j] = minVal3(
                        D[i-1][j],
                        D[i][j-1],
                        D[i-1][j-1]
                    ) + 1;
                }
            }
            
        }
    }
    
    return D[M][N];
}

In [12]:
std::cout << Damerau_Levenstain("Hello", "Hlelo") << std::endl;

1


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fa8b3d476e0


In [13]:
std::cout << Levenstain("Hello", "Hlelo") << std::endl;

2


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fa8b3d476e0


Как можно заметить, расстояние Дамерау-Левенштейна будет меньше, если разница всего лишь в транспозиции.